In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import chainer.functions as F
import chainer.links as L
from chainer import optimizers
from chainer import Variable
from chainer import Chain
import data

In [3]:
mnist = data.load_mnist_data()
x_all = mnist["data"].astype(np.float32) / 255
y_all = mnist["target"].astype(np.int32)
x_train, x_test = np.split(x_all, [60000])
y_train, y_test = np.split(y_all, [60000])
x_train = x_train.reshape((len(x_train), 1, 28,28))
x_test = x_test.reshape((len(x_test), 1, 28,28))

In [4]:
class Model(Chain):
    def __init__(self):
        super(Model, self).__init__(
            conv1=L.Convolution2D(1, 20, 5),
            conv2=L.Convolution2D(20, 50, 5),
            fc1=L.Linear(800, 500),
            fc2=L.Linear(500, 10) )
    def __call__(self, x, train=True):
        cv1 = self.conv1(x)
        relu = F.relu(cv1)
        h = F.max_pooling_2d(relu, 2)
        h = F.max_pooling_2d(F.relu(self.conv2(h)),  2)
        h = F.dropout(F.relu(self.fc1(h)), train=train)
        return self.fc2(h)

In [5]:
model = L.Classifier(Model())
optimizer = optimizers.MomentumSGD(lr=0.01, momentum=0.9)
optimizer.setup(model)

In [6]:
batchsize = 100
datasize = 60000

In [ ]:
for i in range(20):
    indexes = np.random.permutation(datasize)
    for i in range(0, datasize, batchsize):
        x = Variable(x_train[indexes[i:i+batchsize]])
        t = Variable(y_train[indexes[i:i+batchsize]])
        model.zerograds()
        loss = model(x,t)
        loss.backward()
        optimizer.update()
    x = Variable(x_test)
    t = Variable(y_test)
    loss = model(x, t)
    print(loss.data)